In [26]:
#!/usr/bin/env python
import tensorflow as tf
import numpy as np
import h5py
import time
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from skimage.transform import radon, iradon
from skimage.measure import compare_ssim
import math
from google.colab import drive    # in ordre to mount drive to colab
drive.mount('/content/gdrive')

gpu_options = tf.GPUOptions(allow_growth=True)
session_config = tf.ConfigProto(log_device_placement=False, gpu_options=gpu_options, allow_soft_placement=True)
sess = tf.Session(config=session_config)

batch_size = 32
learning_rate = 1.0e-4
width = 256
patch_width = 64
lambda_sl = 0.5
use_gan = True # change to False if not using GAN (LS_GAN).
if use_gan:
  disc_iters = 4
  lambda_al = 0.0025
else:
  disc_iters = 0
  lambda_al = 0

num_epoch = 10

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def read_data(file_name):
    f = h5py.File(file_name, 'r')         # read dataset
    label = np.array(f['data'])           
    f.close()
    return label                 # return data

In [0]:
def two_dim_conv(batch, name, n_filter):
    c = tf.layers.conv2d(
        inputs=batch,
        filters=n_filter,
        kernel_size=[3, 3],
        strides=1,
        padding='same',
        kernel_initializer=tf.contrib.layers.xavier_initializer(),
        use_bias=True,
        activation=None,
        name=name,
        reuse=tf.AUTO_REUSE
    )
    #print("{}:{}".format(name, c.get_shape()))
    return c

In [0]:
def U_res_block_layer(batch, name, n_out):
    conv_1 = two_dim_conv(batch, name + 'res_conv1', n_out)
    conv_1 = tf.nn.relu(conv_1)

    conv_2 = two_dim_conv(conv_1, name + 'res_conv2', n_out)
    conv_2 = tf.nn.relu(conv_2)

    conv_3 = two_dim_conv(conv_2, name + 'res_conv3', n_out)
    block_res = tf.nn.relu(conv_3) + batch

    return block_res

In [0]:
def generator(x, padding='Valid'):
    outputs1 = tf.layers.conv2d(x, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv1', use_bias=True)
    outputs2 = tf.nn.relu(outputs1)
    # 2-D convolutional layers
    outputs2 = U_res_block_layer(outputs2, "res1", 32)
    

    outputs2 = tf.layers.conv2d(outputs2, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv2', use_bias=False)
    outputs3 = tf.nn.relu(outputs2)
    outputs3 = U_res_block_layer(outputs3, "res2", 32)
    

    outputs3 = tf.layers.conv2d(outputs3, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv3', use_bias=False)
    outputs4 = tf.nn.relu(outputs3)
    outputs4 = U_res_block_layer(outputs4, "res3", 32)

    outputs4 = tf.layers.conv2d(outputs4, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv4', use_bias=False)
    outputs5 = tf.nn.relu(outputs4)
    outputs5 = U_res_block_layer(outputs5, "res4", 32)
    
    outputs5 = tf.layers.conv2d_transpose(outputs5, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='deconv5', 
                                          use_bias=False)
    # transpose 2-D convolutional layers
    outputs5 = tf.nn.relu(outputs5)
    outputs5 = U_res_block_layer(outputs5, "res5", 32)
    outputs5 = tf.concat([outputs3, outputs5], -1)

    outputs6 = tf.layers.conv2d_transpose(outputs5, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='deconv6', use_bias=False)
    outputs6 = tf.nn.relu(outputs6)    
    outputs6 = U_res_block_layer(outputs6, "res6", 32)
    outputs6 = tf.concat([outputs2, outputs6], -1)

    outputs7 = tf.layers.conv2d_transpose(outputs6, 32, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='deconv7', use_bias=False)
    outputs7 = tf.nn.relu(outputs7)
    outputs7 = U_res_block_layer(outputs7, "res7", 32)
    outputs7 = tf.concat([outputs1, outputs7], -1)

    outputs8 = tf.layers.conv2d_transpose(outputs7, 1, 3, padding=padding, kernel_initializer=tf.contrib.layers.xavier_initializer(), name='deconv8', use_bias=False)
    outputs = tf.nn.relu(outputs8)
    outputs = U_res_block_layer(outputs, "res8", 1)
    
    return outputs 


In [0]:
def discriminator(x):  
    outputs = tf.layers.conv2d(x, 64, 3, padding='SAME', kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv1')
    outputs = tf.nn.relu(outputs)
        
    outputs = tf.layers.conv2d(outputs, 64, 3, padding='SAME', strides=(2,2), kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv2')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs, 128, 3, padding='SAME', kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv3')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs, 128, 3, padding='SAME', strides=(2,2), kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv4')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs, 256, 3, padding='SAME', kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv5')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.conv2d(outputs, 256, 3, padding='SAME', strides=(2,2), kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv6')
    outputs = tf.nn.relu(outputs)
    
    outputs = tf.layers.dense(outputs, units=1024, name='dense1')
    outputs = tf.nn.relu(outputs)
    outputs = tf.layers.dense(outputs, units=1, name='dense2')
    return outputs

In [32]:
lr = tf.placeholder(dtype=tf.float32, shape=[])  # learning rate
real_placeholder = tf.placeholder(dtype=tf.float32, shape=[batch_size, patch_width, patch_width, 1], name='real')        #real images
fake_placeholder = tf.placeholder(dtype=tf.float32, shape=[batch_size, patch_width, patch_width, 1], name='fake')        #fake images
fake_placeholder_test = tf.placeholder(dtype=tf.float32, shape=[1, width, width, 1], name='fake_test')                   #fakeing tesing images
with tf.variable_scope('generator_model', reuse=tf.AUTO_REUSE) as scope_generator_model:
    Gz = generator(fake_placeholder)                 # feed patches for training
    Gz_test = generator(fake_placeholder_test)       # feed whole images for testing

with tf.variable_scope('discriminator_model', reuse=tf.AUTO_REUSE) as scope_discriminator_model:
    FD_d = discriminator(real_placeholder)           # feed real images into discriminator network
    LD_d = discriminator(Gz)                         # feed fake images into discriminator network


disc_loss = tf.reduce_mean((FD_d - 1)**2 + (LD_d - 0)**2)       # loss function for LS-GAN
disc_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='discriminator_model')
d_trainer = tf.train.AdamOptimizer(learning_rate=lr).minimize(disc_loss, var_list=disc_variables) # training discriminator

ssim = tf.squeeze(tf.reduce_mean(tf.image.ssim(Gz, real_placeholder, 1.0)))
ssim_loss = 1 - ssim
mse = tf.reduce_mean(tf.squared_difference(Gz, real_placeholder))
gen_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='generator_model')
gen_cost = tf.reduce_mean((LD_d - 1)**2)          # GAN loss for generator

g_trainer = tf.train.AdamOptimizer(learning_rate=lr).minimize(mse + lambda_al * gen_cost + lambda_sl * ssim_loss, var_list=gen_variables) # training generator

saver = tf.train.Saver()
print("enter Session")
sess.run(tf.global_variables_initializer())       # initialize parameters
#saver.restore(sess, '/content/gdrive/My Drive/hands_on4/Autoencoder/SIGmathmodel100.ckpt')  # use to restore model
print('model restored')
print("Begin training")
label = read_data('/content/gdrive/My Drive/hands_on4/Autoencoder/Train/Label.hdf5') # read label
data = read_data('/content/gdrive/My Drive/hands_on4/Autoencoder/Train/Data.hdf5') # read data


for iteration in range(num_epoch):
    #break
    label, data = shuffle(label, data)  # shuffling data & label, same as last time
    val_lr = learning_rate / (iteration + 1)       # learning rate is decreased
    num_batches = data.shape[0] // batch_size      # how many batches?
    for i in range(num_batches):
        for _ in range(disc_iters): # disc_iters will be set as 0 if you de-activate GAN
            idx = np.random.permutation(label.shape[0])       # the input to the discriminator and the input to the generator should be different
            batch_label = label[idx[:batch_size]]
            batch_label = np.expand_dims(batch_label, -1)
            
            batch_data = data[idx[:batch_size]]
            batch_data = np.expand_dims(batch_data, -1)
            
            _ = sess.run(d_trainer, feed_dict={real_placeholder: batch_label, # training discriminator
                                               fake_placeholder: batch_data,
                                               lr: val_lr}) 
            
        batch_label = label[i * batch_size: (i + 1) * batch_size] # the input to the discriminator and the input to the generator should be different
        batch_label = np.expand_dims(batch_label, -1)
        batch_data = data[i * batch_size: (i + 1) * batch_size]
        batch_data = np.expand_dims(batch_data, -1)
        _mse, _ssim, _ = sess.run([mse, ssim, g_trainer], feed_dict={real_placeholder: (batch_label),  # training generator
                                                                             fake_placeholder: (batch_data), lr: val_lr})

        print('Epoch: %d - %d - mse %.6f: - ssim %.6f: ' % (iteration, i, _mse, _ssim))

saver.save(sess, '/content/gdrive/My Drive/hands_on4/Autoencoder/SIGmathmodel100.ckpt')  # save model
  
print("testing dataset")
test_label = read_data('/content/gdrive/My Drive/hands_on4/Autoencoder/Test/Label.hdf5')   # reading testing label
test_data = read_data('/content/gdrive/My Drive/hands_on4/Autoencoder/Test/Data.hdf5')   # reaing testing data
for c in range(test_label.shape[0]):
    test_batch_label = test_label[c * 1: (c + 1) * 1]  # feed images one by one
    test_batch_label = np.expand_dims(test_batch_label, -1)
    
    test_batch_data = test_data[c * 1: (c + 1) * 1]
    test_batch_data = np.expand_dims(test_batch_data, -1)
    
    with tf.variable_scope('generator_model') as scope:
        scope.reuse_variables()
        estimated = sess.run(Gz_test, feed_dict={fake_placeholder_test: test_batch_data})      # testing
        t_r = np.squeeze(test_batch_label)
        e_1 = np.squeeze(estimated)
        t_o = np.squeeze(test_batch_data)
        f_n = '/content/gdrive/My Drive/hands_on4/Autoencoder/testing_results/' + str(c) + 'validation.jpg' #save images
        plt.imsave(f_n, e_1, cmap='gray')
        f_n = '/content/gdrive/My Drive/hands_on4/Autoencoder/testing_results/' + str(c) + 'real_.jpg'
        plt.imsave(f_n, t_r, cmap='gray')
        f_n = '/content/gdrive/My Drive/hands_on4/Autoencoder/testing_results/' + str(c) + 'original_.jpg'
        plt.imsave(f_n, t_o, cmap='gray')

        z = (t_r - e_1) ** 2
        print(np.mean(z))   # print mse loss.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
enter Session
model restored
Begin training
Epoch: 0 - 0 - mse 0.011783: - ssim 0.806754: 
Epoch: 0 - 1 - mse 0.013204: - ssim 0.777343: 
Epoch: 0 - 2 - mse 0.007774: - ssim 0.840091: 
Epoch: 0 - 3 - mse 0.006061: - ssim 0.865351: 
Epoch: 0 - 4 - mse 0.005497: - ssim 0.884312: 
Epoch: 0 - 5 - mse 0.003991: - ssim 0.901705: 
Epoch: 0 - 6 - mse 0.004053: - ssim 0.903627: 
Epoch: 0 - 7 - mse 0.005517: - ssim 0.891854: 
Epoch: 0 - 8 - mse 0.003448: - ssim 0.923909: 
Epoch: 0 - 9 - mse 0.003692: - ssim 0.921499: 
Epoch: 0 - 10 - mse 0.003973: - ssim 0.903666: 
Epoch: 0 - 11 - mse 0.003400: - ssim 0.929904: 
Epoch: 0 - 12 - mse 0.003254: - ssim 0.926852: 
Epoch: 0 - 13 - mse 0.002584: - ssim 0.928867: 
Epoch: 0 - 14 - mse 0.003384: - ssim 0.929584: 
Epoch: 0 - 15 - mse 0.003493: - ssim 0.928930: 
Epoch: 0 - 16 - mse 0.003459: - ssim 0.932453